<a href="https://colab.research.google.com/github/eunicedai/Integrative-Analyze/blob/master/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Homework 1** 

> TaiHsuan Wu | 2524355409



1.a.) 
How would the decisions change if the first half of the regular loom hours used were at the rate provided in Table 1, but the rest of the usage is charged at a rate which is 33% higher?

> After change the half time of regular loom rate to 33% higher, the effieciency of regular loom would be higher. In this case, the company would produce by using regular loom more than others. 

1.b.) Using the MIP formulation (1-12) in the paper, and using the modifications suggested in part a), solve the revised model.  Considering the conclusions of alternative optima for the model reported in the paper, please address the same question for the revised model.

In [0]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && 
  bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

In [0]:
!pip install -q pyomo

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
from __future__ import division
from pyomo.environ import *

In [0]:
data = {
    1: {'demand': 16500.0, 'drate': 4.653, 'rrate': 0, 'mcost': 0.66, 'ocost': 0.80},
    2: {'demand': 52000.0, 'drate': 4.653, 'rrate': 0, 'mcost': 0.56, 'ocost': 0.70},
    3: {'demand': 45000.0, 'drate': 4.653, 'rrate': 0, 'mcost': 0.66, 'ocost': 0.85},
    4: {'demand': 22000.0, 'drate': 4.653, 'rrate': 0, 'mcost': 0.55, 'ocost': 0.70},
    5: {'demand': 76500.0, 'drate': 5.194, 'rrate': 5.194, 'mcost': 0.61, 'ocost': 0.75},
    6: {'demand': 110000.0,'drate': 3.809, 'rrate': 3.809, 'mcost': 0.62, 'ocost': 0.75},
    7: {'demand': 122000.0,'drate': 4.185, 'rrate': 4.185, 'mcost': 0.65, 'ocost': 0.80},
    8: {'demand': 62000.0, 'drate': 5.232, 'rrate': 5.232, 'mcost': 0.49, 'ocost': 0.60},
    9: {'demand': 7500.0,  'drate': 5.232, 'rrate': 5.232, 'mcost': 0.50, 'ocost': 0.70},
    10:{'demand': 69000.0, 'drate': 5.232, 'rrate': 5.232, 'mcost': 0.44, 'ocost': 0.60},
    11:{'demand': 70000.0, 'drate': 3.733, 'rrate': 3.733, 'mcost': 0.64, 'ocost': 0.80},
    12:{'demand': 82000.0, 'drate': 4.185, 'rrate': 4.185, 'mcost': 0.57, 'ocost': 0.75},
    13:{'demand': 10000.0, 'drate': 4.439, 'rrate': 4.439, 'mcost': 0.50, 'ocost': 0.65},
    14:{'demand': 380000.0,'drate': 5.232, 'rrate': 5.232, 'mcost': 0.31, 'ocost': 0.45},
    15:{'demand': 62000.0, 'drate': 4.185, 'rrate': 4.185, 'mcost': 0.50, 'ocost': 0.70},
}

In [7]:
# display feed information
print(pd.DataFrame.from_dict(data).T)

      demand  drate  rrate  mcost  ocost
1    16500.0  4.653  0.000   0.66   0.80
2    52000.0  4.653  0.000   0.56   0.70
3    45000.0  4.653  0.000   0.66   0.85
4    22000.0  4.653  0.000   0.55   0.70
5    76500.0  5.194  5.194   0.61   0.75
6   110000.0  3.809  3.809   0.62   0.75
7   122000.0  4.185  4.185   0.65   0.80
8    62000.0  5.232  5.232   0.49   0.60
9     7500.0  5.232  5.232   0.50   0.70
10   69000.0  5.232  5.232   0.44   0.60
11   70000.0  3.733  3.733   0.64   0.80
12   82000.0  4.185  4.185   0.57   0.75
13   10000.0  4.439  4.439   0.50   0.65
14  380000.0  5.232  5.232   0.31   0.45
15   62000.0  4.185  4.185   0.50   0.70


In [0]:
!apt-get install -y -qq glpk-utils

In [0]:
def table1_m2(data):
  Ky = data.keys()
  model = ConcreteModel()
  model.r1 = Var(RangeSet(15), domain=NonNegativeReals)
  model.r2 = Var(RangeSet(15), domain=NonNegativeReals)
  model.d = Var(RangeSet(15), domain=NonNegativeReals)
  model.o = Var(RangeSet(15), domain=NonNegativeReals)
  model.obj = Objective(expr = (sum(data[i]['mcost']*(model.r1[i]+model.r2[i]+model.d[i]) for i in Ky)+
                                sum(model.o[i]*data[i]['ocost'] for i in Ky)), 
                        sense = minimize)
  model.limits = ConstraintList()
  model.limits.add(sum(model.r1[i]/data[i]['rrate']*2 for i in Ky if data[i]['rrate'] != 0) <= 196560)
  model.limits.add(sum(model.d[i]/data[i]['drate'] for i in Ky) <= 32760)
  for i in Ky:
    model.limits.add(model.r1[i]+model.r2[i]+model.d[i]+model.o[i] == data[i]['demand'])
    if i > 4:
      model.limits.add(model.r1[i]/data[i]['rrate'] == model.r2[i]/(data[i]['rrate']*1.33))
  for i in range(1,5):
    model.limits.add(model.r1[i] == 0)
    model.limits.add(model.r2[i] == 0)

  # solve
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

  res = pd.DataFrame()
  for i in Ky:
    res.loc[i, 'Fabric'] = i
    res.loc[i, 'Dobby'] = model.d[i]()
    res.loc[i, 'Regular'] = model.r1[i]()+model.r2[i]()
    res.loc[i, 'Outsource'] = model.o[i]()
    res.loc[i, 'Total'] = (res.loc[i, 'Dobby']+res.loc[i, 'Regular']+res.loc[i, 'Outsource'])
  
  print(res)
  print(model.obj())

In [37]:
table1_m2(data)

    Fabric    Dobby   Regular  Outsource     Total
1      1.0  16500.0       0.0        0.0   16500.0
2      2.0  52000.0       0.0        0.0   52000.0
3      3.0  45000.0       0.0        0.0   45000.0
4      4.0  22000.0       0.0        0.0   22000.0
5      5.0      0.0   76500.0        0.0   76500.0
6      6.0      0.0  110000.0        0.0  110000.0
7      7.0      0.0  122000.0        0.0  122000.0
8      8.0      0.0   62000.0        0.0   62000.0
9      9.0      0.0    7500.0        0.0    7500.0
10    10.0      0.0   69000.0        0.0   69000.0
11    11.0      0.0   70000.0        0.0   70000.0
12    12.0      0.0   82000.0        0.0   82000.0
13    13.0      0.0   10000.0        0.0   10000.0
14    14.0      0.0  380000.0        0.0  380000.0
15    15.0      0.0   62000.0        0.0   62000.0
585805.0


In [0]:
def table2_m2(data):
  Ky = data.keys()
  model = ConcreteModel()
  model.r1 = Var(RangeSet(15), domain=NonNegativeIntegers)
  model.r2 = Var(RangeSet(15), domain=NonNegativeIntegers)
  model.d = Var(RangeSet(15), domain=NonNegativeIntegers)
  model.o = Var(RangeSet(15), domain=NonNegativeIntegers)
  model.R = Var(RangeSet(15), within=Binary)
  model.O = Var(RangeSet(15), within=Binary)
  model.D = Var(RangeSet(15), within=Binary)
  model.obj = Objective(expr = (sum(data[i]['mcost']*(model.r1[i]+model.r2[i]+model.d[i]) for i in Ky)+ 
                                sum(model.o[i]*data[i]['ocost'] for i in Ky)), 
                        sense = minimize)
  model.limits = ConstraintList()
  model.limits.add(sum(model.r1[i]/data[i]['rrate']*2 for i in Ky if data[i]['rrate'] != 0) <= 196560)
  model.limits.add(sum(model.d[i]/data[i]['drate'] for i in Ky) <= 32760)
  for i in Ky:
    model.limits.add(model.r1[i]+model.r2[i]+model.d[i]+model.o[i] == data[i]['demand'])
    model.limits.add(model.r1[i]+model.r2[i] <= data[i]['demand']*model.R[i])
    model.limits.add(model.o[i] <= data[i]['demand']*model.O[i])
    model.limits.add(model.d[i] <= data[i]['demand']*model.D[i])
    model.limits.add(model.R[i]+model.O[i]+model.D[i] == 1)
    if i > 4:
      model.limits.add(model.r1[i]/data[i]['rrate'] == model.r2[i]/(data[i]['rrate']*1.33))
  for i in range(1,5):
    model.limits.add(model.r1[i] == 0)
    model.limits.add(model.r2[i] == 0)
  # solve
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

  res = pd.DataFrame()
  for i in Ky:
    res.loc[i, 'Fabric'] = i
    res.loc[i, 'Dobby'] = model.d[i]()
    res.loc[i, 'Regular'] = model.r1[i]()+model.r2[i]()
    res.loc[i, 'Outsource'] = model.o[i]()
    res.loc[i, 'Total'] = (res.loc[i, 'Dobby']+res.loc[i, 'Regular']+res.loc[i, 'Outsource'])
  
  print(res)
  print(model.obj())

In [43]:
table2_m2(data)

    Fabric    Dobby  Regular  Outsource     Total
1      1.0      0.0      0.0    16500.0   16500.0
2      2.0      0.0      0.0    52000.0   52000.0
3      3.0      0.0      0.0    45000.0   45000.0
4      4.0  22000.0      0.0        0.0   22000.0
5      5.0      0.0      0.0    76500.0   76500.0
6      6.0      0.0      0.0   110000.0  110000.0
7      7.0      0.0      0.0   122000.0  122000.0
8      8.0      0.0      0.0    62000.0   62000.0
9      9.0      0.0      0.0     7500.0    7500.0
10    10.0  69000.0      0.0        0.0   69000.0
11    11.0      0.0      0.0    70000.0   70000.0
12    12.0      0.0      0.0    82000.0   82000.0
13    13.0      0.0      0.0    10000.0   10000.0
14    14.0      0.0      0.0   380000.0  380000.0
15    15.0  62000.0      0.0        0.0   62000.0
736235.0


After adding the constraint of binary, the objective value is worse than without binary. Thus use the result from first model for alternative optimal.

In [0]:
def table3_m2(data):
  Ky = data.keys()
  model = ConcreteModel()
  model.r1 = Var(RangeSet(15), domain=NonNegativeReals)
  model.r2 = Var(RangeSet(15), domain=NonNegativeReals)
  model.d = Var(RangeSet(15), domain=NonNegativeReals)
  model.o = Var(RangeSet(15), domain=NonNegativeReals)

  model.obj = Objective(expr = (sum(model.d[i]+model.r1[i]+model.r2[i]+model.o[i] for i in Ky)), 
                        sense = maximize)
  model.limits = ConstraintList()
  model.limits.add(sum(model.r1[i]/data[i]['rrate']*2 for i in Ky if data[i]['rrate'] != 0) <= 196560)
  model.limits.add(sum(model.d[i]/data[i]['drate'] for i in Ky) <= 32760)
  for i in Ky:
    model.limits.add(model.r1[i]+model.r2[i]+model.d[i]+model.o[i] == data[i]['demand'])
    if i > 4:
      model.limits.add(model.r1[i]/data[i]['rrate'] == model.r2[i]/(data[i]['rrate']*1.33))
  for i in range(1,5):
    model.limits.add(model.r1[i] == 0)
    model.limits.add(model.r2[i] == 0)
  model.limits.add(sum(data[i]['mcost']*(model.r1[i]+model.r2[i]+model.d[i]) for i in Ky)+
                   sum(model.o[i]*data[i]['ocost'] for i in Ky) == 585805.0)

  # solve
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

  res = pd.DataFrame()
  for i in Ky:
    res.loc[i, 'Fabric'] = i
    res.loc[i, 'Dobby'] = model.d[i]()
    res.loc[i, 'Regular'] = model.r1[i]()+model.r2[i]()
    res.loc[i, 'Outsource'] = model.o[i]()
    res.loc[i, 'Total'] = (res.loc[i, 'Dobby']+res.loc[i, 'Regular']+res.loc[i, 'Outsource'])
  
  print(res)

In [103]:
table3_m2(data)

    Fabric    Dobby   Regular  Outsource     Total
1      1.0  16500.0       0.0        0.0   16500.0
2      2.0  52000.0       0.0        0.0   52000.0
3      3.0  45000.0       0.0        0.0   45000.0
4      4.0  22000.0       0.0        0.0   22000.0
5      5.0      0.0   76500.0        0.0   76500.0
6      6.0      0.0  110000.0        0.0  110000.0
7      7.0      0.0  122000.0        0.0  122000.0
8      8.0      0.0   62000.0        0.0   62000.0
9      9.0      0.0    7500.0        0.0    7500.0
10    10.0      0.0   69000.0        0.0   69000.0
11    11.0      0.0   70000.0        0.0   70000.0
12    12.0      0.0   82000.0        0.0   82000.0
13    13.0      0.0   10000.0        0.0   10000.0
14    14.0      0.0  380000.0        0.0  380000.0
15    15.0      0.0   62000.0        0.0   62000.0


After comparing the table with first table, we noticed that the optimal solutions are the same they both have optimal solution of 585805.0).

1.c.)  Using the Ohio counties population data available at http://www.ohio-demographics.com/counties_by_population solve the model provided in model formulation (25-29) , for k=15, and compare your decisions with those reported on page 17 of the paper. 

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [0]:
ohio = pd.read_csv('gdrive/My Drive/ISE533/ohio_sheet.csv')

In [71]:
ohio.head()

,County,Population,Map_Label,Adjacent Matrix
0,Williams County,"36,804",1,"2,18,19"
1,Defiance County,"38,165",2,"1, 3,18,17"
2,Paulding County,"18,760",3,"2,4,17"
3,Van Wert County,"28,281",4,"3, 5, 15, 16, 17"
4,Mercer County,"40,959",5,"4,6,15"


In [0]:
country = {}

for i in ohio.index:
  ohio.loc[i]['Adjacent Matrix'].replace(" ", "")
  country[i+1] = [int(x) for x in ohio.loc[i]['Adjacent Matrix'].split(",")]
  country[i+1].append(i+1)

In [0]:
def adj_country(data, matrix):
  model = ConcreteModel()
  model.x = Var(range(88), within=Binary)
  model.y = Var(range(88), within=Binary)
  model.obj = Objective(expr = sum(int(data.loc[i]['Population'].replace(",", ""))*model.y[i] for i in data.index), 
                        sense = maximize)
  model.limits = ConstraintList()
  model.limits.add(sum(model.x[i] for i in range(0,88)) <= 15)
  for i in range(0,88):
    model.limits.add(sum(model.x[j] for j in range(0,88) if (j+1) in matrix[i+1]) >= model.y[i])
  
  # solve
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

  res = pd.DataFrame()
  for i in data.index:
    if model.x[i]() == 1:
      res.loc[i+1, 'Country'] = data.loc[i]['County']
      res.loc[i+1, 'Population'] = data.loc[i]['Population']
      res.loc[i+1, 'Principal'] = 1
  
  print(model.obj())
  print(res)

In [100]:
adj_country(ohio, country)

11689442.0
            Country Population  Principal
4   Van Wert County     28,281        1.0
8     Butler County    382,378        1.0
14    Shelby County     48,627        1.0
18     Henry County     27,086        1.0
29     Brown County     43,602        1.0
33   Madison County     44,413        1.0
35    Marion County     65,256        1.0
38  Sandusky County     58,799        1.0
51   Jackson County     32,384        1.0
55   Licking County    175,769        1.0
59   Ashland County     53,745        1.0
68    Athens County     65,818        1.0
75     Stark County    371,574        1.0
78    Geauga County     94,031        1.0
87   Belmont County     67,505        1.0


The result is the same as the one in the paper (the second alternative solution).